In [1]:
ls

drive/  sample_data/


In [2]:
cd  drive/MyDrive

/content/drive/MyDrive


In [3]:
cd DIC

/content/drive/MyDrive/DIC


In [4]:
ls

 ADIC2D.ipynb         dist.pkl                               'Sample3 Reference1.tif'
 calibration.pkl      EDSR_x4.pt                             'Sample3 Reference2.tif'
 CameraCalibration/   images/                                'Sample3 Reference.tif'
 cameraMatrix.pkl     keyPointExtraction.ipynb                speckle_pattern.jpg
 correctedImage/     'Sample3-000 X0.00 Y0.00 N2 C0 R0.tif'


In [40]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

# Support Functions
def STD(errin, maer):
    err = errin[~np.isnan(errin)]
    total = np.sum((np.abs(err) - maer) ** 2)
    count = len(err)
    return np.sqrt(total / (count - 1))

def RMSE_calc(err):
    return np.sqrt(np.mean(err ** 2))

def ImageNamesLoad(Sample):
    ImageLocation = os.getcwd()
    if Sample == 1:
        ImName = [os.path.join(ImageLocation, f'Sample1/trxy_s2_{i:02d}.tif') for i in range(21)]
    elif Sample == 2:
        ImName = [os.path.join(ImageLocation, f'DIC Challenge 1.0/Sample2/trs2_b8_{i:02d}.tif') for i in range(21)]
    elif Sample == 3:
        ImName = [
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3 Reference.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-000 X0.00 Y0.00 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-001 X0.10 Y0.10 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-002 X0.20 Y0.20 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-003 X0.30 Y0.30 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-004 X0.40 Y0.40 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-005 X0.50 Y0.50 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-006 X0.60 Y0.60 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-007 X0.70 Y0.70 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-008 X0.80 Y0.80 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-009 X0.90 Y0.90 N2 C0 R0.tif'),
            os.path.join(ImageLocation, 'correctedImage/Sample3/Sample3-010 X1.00 Y1.00 N2 C0 R0.tif')
        ]
    elif Sample == 4:
        ImName = [
            os.path.join(ImageLocation, 'correctedImage/FilteredImage/refrence_image0.tif')
        ] + [
            os.path.join(ImageLocation, f'correctedImage/FilteredImage/image{i}.tif') for i in range(1, 12)
        ]
    return ImName


In [141]:
# Main function equivalent to runme.m
def run_analysis():
    # print("1")
    Sample = 3  # which sample to analyze
    FileNames = ImageNamesLoad(Sample)  # load appropriate images for the chosen sample
    print(FileNames[0])
    Mask = np.ones(cv2.imread(FileNames[0], cv2.IMREAD_GRAYSCALE).shape)  # define a mask that includes all pixels of the image

    # Function inputs (which set up the DIC analysis)
    GaussFilter = [0.4, 5]
    StepSize = 5
    SubSize = 41
    SubShape = 'Circle'
    SFOrder = 1
    RefStrat = 0
    StopCritVal = 1e-4
    WorldCTs = 0
    ImageCTs = 0
    rho = 0

    # Perform DIC analysis
    ProcData = ADIC2D(FileNames, Mask, GaussFilter, StepSize, SubSize, SubShape, SFOrder, RefStrat, StopCritVal, WorldCTs, ImageCTs, rho)

    # Correct displacements of incremental reference image strategy so that they describe displacement relative to the first image of the image set
    if RefStrat == 1:
        for d in range(1, len(ProcData)):
            ProcData[d]['Uw'] += ProcData[d-1]['Uw']

    # ProcData = AddGridFormat(ProcData)  # add gridded matrices for display purposes

    return ProcData

ProcData = run_analysis()


/content/drive/MyDrive/DIC/correctedImage/Sample3/Sample3 Reference.tif
images /content/drive/MyDrive/DIC/correctedImage/Sample3/Sample3-000 X0.00 Y0.00 N2 C0 R0.tif


KeyboardInterrupt: 

In [125]:
import numpy as np
import cv2

def ADIC2D(FileNames, Mask, GaussFilter, StepSize, SubSize, SubShape, SFOrder, RefStrat, StopCritVal, WorldCTs, ImgCTs, rho):
    # print("2")
    n = len(FileNames)
    r, c = cv2.imread(FileNames[0], cv2.IMREAD_GRAYSCALE).shape

    XosX, XosY = np.meshgrid(
        np.arange((SubSize+1)/2 + StepSize, c-(SubSize+1)/2, StepSize),
        np.arange((SubSize+1)/2 + StepSize, r-(SubSize+1)/2, StepSize)
    )
    Xos = np.vstack((XosX.ravel(), XosY.ravel()))

    valid_indices = [
        i for i in range(Xos.shape[1]) if np.min(Mask[int(Xos[1, i])-SubSize//2:int(Xos[1, i])+SubSize//2+1,
                                                      int(Xos[0, i])-SubSize//2:int(Xos[0, i])+SubSize//2+1])
    ]
    Xos = Xos[:, valid_indices]

    ProcData = []
    for i in range(n):
        ProcData.append({
            'ImgName': FileNames[i],
            'ImgSize': (r, c),
            'ImgFilt': GaussFilter,
            'SubSize': SubSize * np.ones(Xos.shape[1]),
            'SubShape': np.full((Xos.shape[1], 1), SubShape),
            'SFOrder': np.full((Xos.shape[1], 1), SFOrder),
            'Xos': Xos,
            'P': np.zeros((12, Xos.shape[1])),
            'C': np.ones(Xos.shape[1]),
            'StopVal': np.ones(Xos.shape[1]) * StopCritVal,
            'Iter': np.zeros(Xos.shape[1]),
        })

    ProcData = ImgCorr(n, ProcData, FileNames, RefStrat, StopCritVal)
    ProcData = CSTrans(n, ProcData, WorldCTs, ImgCTs, rho)

    return ProcData


In [101]:
for i, val in enumerate((ProcData[0]['Xos'])[:10]):
  print(val)


[ 26.  31.  36. ... 476. 481. 486.]
[ 26.  26.  26. ... 486. 486. 486.]


In [124]:
def AddGridFormat(PD):
    #print("3")
    for d in range(len(PD)):
        XindicesUnique = np.sort(np.unique(np.ceil(PD[d]['Xos'][0, :])))
        Xindices = np.full(int(XindicesUnique[-1]), np.nan)
        Xindices[XindicesUnique.astype(int)] = np.arange(len(XindicesUnique))

        YindicesUnique = np.sort(np.unique(np.ceil(PD[d]['Xos'][1, :])))
        Yindices = np.full(int(YindicesUnique[-1]), np.nan)
        Yindices[YindicesUnique.astype(int)] = np.arange(len(YindicesUnique))

        CheckElements = np.zeros((len(Yindices), len(Xindices)))

        for q in range(PD[d]['Xow'].shape[1]):
            y_idx = int(np.ceil(PD[d]['Xos'][1, q]))
            x_idx = int(np.ceil(PD[d]['Xos'][0, q]))

            PD[d]['POSX'][int(Yindices[y_idx]), int(Xindices[x_idx])] = PD[d]['Xow'][0, q]
            PD[d]['POSY'][int(Yindices[y_idx]), int(Xindices[x_idx])] = PD[d]['Xow'][1, q]
            PD[d]['UX'][int(Yindices[y_idx]), int(Xindices[x_idx])] = PD[d]['Uw'][0, q]
            PD[d]['UY'][int(Yindices[y_idx]), int(Xindices[x_idx])] = PD[d]['Uw'][1, q]
            CheckElements[int(Yindices[y_idx]), int(Xindices[x_idx])] = PD[d]['Xos'][0, q]

        for i in range(int(PD[d]['POSX'].shape[0])):
            for j in range(int(PD[d]['POSX'].shape[1])):
                if CheckElements[i, j] == 0:
                    PD[d]['POSX'][i, j] = np.nan
                    PD[d]['POSY'][i, j] = np.nan
                    PD[d]['UX'][i, j] = np.nan
                    PD[d]['UY'][i, j] = np.nan

    return PD


In [108]:
 def SubExtract(Mat, Xos, SubSize):
    return Mat[int(Xos[1]-(SubSize-1)/2):int(Xos[1]+(SubSize-1)/2)+1,
                   int(Xos[0]-(SubSize-1)/2):int(Xos[0]+(SubSize-1)/2)+1]

In [123]:
import numpy as np
import cv2
from scipy.interpolate import RegularGridInterpolator
from time import time

def ImgCorr(n, PD, ImNames, RefStrat, StopCritVal):
    #print("5")

    for d in range(1, n):  # outer loop
        print(f"images {ImNames[d]}")
        tic = time()
        G = cv2.imread(ImNames[d], cv2.IMREAD_GRAYSCALE).astype(float) / 255.0

        if all(PD[d]['ImgFilt']):
            G = cv2.GaussianBlur(G, (int(PD[d]['ImgFilt'][1]), int(PD[d]['ImgFilt'][1])), PD[d]['ImgFilt'][0])

        # Correct setup for RegularGridInterpolator
        y = np.arange(G.shape[0])
        x = np.arange(G.shape[1])
        InterpCoef = RegularGridInterpolator((y, x), G, method='cubic')

        if RefStrat == 1 or d == 1:
            F = cv2.imread(ImNames[d-1], cv2.IMREAD_GRAYSCALE).astype(float) / 255.0
            if all(PD[d]['ImgFilt']):
                F = cv2.GaussianBlur(F, (int(PD[d]['ImgFilt'][1]), int(PD[d]['ImgFilt'][1])), PD[d]['ImgFilt'][0])

            dFdx = cv2.Sobel(F, cv2.CV_64F, 1, 0, ksize=3)
            dFdy = cv2.Sobel(F, cv2.CV_64F, 0, 1, ksize=3)

            PD[d]['Xos'][0, :] = PD[d-1]['Xos'][0, :] + np.fix(PD[d-1]['P'][0, :])
            PD[d]['Xos'][1, :] = PD[d-1]['Xos'][1, :] + np.fix(PD[d-1]['P'][6, :])

            # Assuming PCM function is defined elsewhere
            PD[d]['P'][0, :], PD[d]['P'][6, :] = np.vectorize(lambda XosX, XosY, SubSize: PCM(F, G, SubSize, XosX, XosY, SubExtract))(
                PD[d]['Xos'][0, :], PD[d]['Xos'][1, :], PD[d]['SubSize'])
        else:
            PD[d]['P'] = PD[d-1]['P']

        P = np.zeros_like(PD[d]['P'])
        C = np.zeros_like(PD[d]['C'])
        Iter = np.zeros_like(PD[d]['C'])
        StopVal = np.ones_like(PD[d]['C'])

        for q in range(PD[d]['Xos'].shape[1]):  # inner loop
            # Assuming SubShapeExtract and SubCorr functions are defined elsewhere
            f, dfdx, dfdy, dX, dY = SubShapeExtract(PD[d]['SubSize'][q], PD[d]['SubShape'][q, :],
                                                    PD[d]['Xos'][:, q], F, dFdx, dFdy, SubExtract)
            P[:, q], C[q], Iter[q], StopVal[q] = SubCorr(InterpCoef, f, dfdx, dfdy, PD[d]['SubSize'][q],
                                                         PD[d]['SFOrder'][q], PD[d]['Xos'][:, q], dX, dY,
                                                         PD[d]['P'][:, q], StopCritVal)

        PD[d]['P'] = P
        PD[d]['C'] = C
        PD[d]['Iter'] = Iter
        PD[d]['StopVal'] = StopVal

        if (d-1) % 10 == 0:
            print('Image/Total| Time (s) | CC (min) | CC(mean) | Iter (max)')
        print(f' {d:4d}/{n:4d} | {time()-tic:7.3f} | {np.min(PD[d]["C"]):8.6f} | {np.nanmean(PD[d]["C"]):8.6f} | {np.max(PD[d]["Iter"]):3.0f}')

    return PD

# Note: PCM, SubShapeExtract, and SubCorr functions need to be defined separately

In [115]:
import numpy as np
from scipy.fft import fft2, ifft2, fftshift

def PCM(F, G, SubSize, XosX, XosY, SubExtract):
    #print("6")
    # Extract subsets from F and G
    F_sub = SubExtract(F, [XosX, XosY], SubSize)
    G_sub = SubExtract(G, [XosX, XosY], SubSize)

    # Compute Normalized Cross Power Spectrum
    F_fft = fft2(F_sub)
    G_fft = fft2(G_sub)
    NCPS = (F_fft * np.conj(G_fft)) / np.abs(F_fft * np.conj(G_fft))

    # Compute Cross-Correlation
    CC = np.abs(ifft2(NCPS))

    # Find the maximum correlation
    vid, uid = np.unravel_index(np.argmax(CC), CC.shape)

    # Compute the shift
    IndShift = -fftshift(np.arange(-SubSize//2, SubSize//2))
    u = IndShift[uid]
    v = IndShift[vid]

    return u, v

# Note: SubExtract function needs to be defined separately

In [116]:
import numpy as np

def SubShapeExtract(SubSize, SubShape, Xos, F, dFdx, dFdy, SubExtract):
    #print("7")
    if SubShape == 'Square':
        f = SubExtract(F, Xos, SubSize).reshape(SubSize*SubSize, 1)
        dfdx = SubExtract(dFdx, Xos, SubSize).reshape(SubSize*SubSize, 1)
        dfdy = SubExtract(dFdy, Xos, SubSize).reshape(SubSize*SubSize, 1)
        dX, dY = np.meshgrid(np.arange(-(SubSize-1)/2, (SubSize-1)/2+1),
                             np.arange(-(SubSize-1)/2, (SubSize-1)/2+1))
        dX = dX.ravel()
        dY = dY.ravel()

    elif SubShape == 'Circle':
        f = SubExtract(F, Xos, SubSize)
        dfdx = SubExtract(dFdx, Xos, SubSize)
        dfdy = SubExtract(dFdy, Xos, SubSize)
        dX, dY = np.meshgrid(np.arange(-(SubSize-1)/2, (SubSize-1)/2+1),
                             np.arange(-(SubSize-1)/2, (SubSize-1)/2+1))
        mask_keep = np.sqrt(np.abs(dX)**2 + np.abs(dY)**2) <= (SubSize/2 - 0.5)
        f = f[mask_keep]
        dfdx = dfdx[mask_keep]
        dfdy = dfdy[mask_keep]
        dX = dX[mask_keep]
        dY = dY[mask_keep]

    return f, dfdx, dfdy, dX, dY


In [140]:
import numpy as np
from scipy.interpolate import griddata

def SubCorr(InterpCoef, f, dfdx, dfdy, SubSize, SFOrder, Xos, dX, dY, P, StopCritVal):
    W, dFdWdP, SFPVec2Mat, Mat2SFPVec, StopCrit = SFExpressions(SFOrder)

    dfdWdP = dFdWdP(dX.flatten(), dY.flatten(), dfdx.flatten(), dfdy.flatten())
    Hinv = np.linalg.inv(dfdWdP.T @ dfdWdP)

    f_bar = np.mean(f)
    f_tilde = np.sqrt(np.sum((f - f_bar)**2))

    flag = 0
    iter = 0
    dP = np.ones_like(P)

    # Prepare grid for griddata
    y, x = InterpCoef.grid
    xx, yy = np.meshgrid(x, y)
    values = InterpCoef.values

    while flag == 0:
        dXY = W(dX.flatten(), dY.flatten(), P)

        points = np.column_stack((Xos[1] + dXY[:, 1], Xos[0] + dXY[:, 0]))

        # Use griddata for interpolation
        g = griddata((yy.flatten(), xx.flatten()), values.flatten(), points, method='cubic', fill_value=np.nan)

        # Handle NaN values
        nan_mask = np.isnan(g)
        if np.any(nan_mask):
            g[nan_mask] = griddata((yy.flatten(), xx.flatten()), values.flatten(), points[nan_mask], method='nearest')

        g_bar = np.mean(g)
        g_tilde = np.sqrt(np.sum((g - g_bar)**2))

        StopVal = StopCrit(dP, (SubSize-1)/2)

        if np.any([StopVal < StopCritVal, iter > 100]):
            flag = 1
            C = 1 - np.sum(((f.flatten()-f_bar)/f_tilde-(g-g_bar)/g_tilde)**2)/2
        else:
            J = dfdWdP.T @ (f.flatten() - f_bar - f_tilde/g_tilde * (g - g_bar))

            if SFOrder == 0:
                dP[[0, 6]] = -Hinv @ J
            elif SFOrder == 1:
                dP[[0, 1, 2, 6, 7, 8]] = -Hinv @ J
            elif SFOrder == 2:
                dP[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]] = -Hinv @ J

            # Safe division
            P_mat = SFPVec2Mat(P)
            dP_mat = SFPVec2Mat(dP)
            epsilon = 1e-10
            P_updated = P_mat / (dP_mat + epsilon)
            P_updated = np.where(np.isfinite(P_updated), P_updated, P_mat)
            P = Mat2SFPVec(P_updated)

        iter += 1

    return P, C, iter, StopVal

In [131]:
import numpy as np

def SFExpressions(SFOrder):
    #print("9")
    if SFOrder == 0:  # Zero order SF
        W = lambda dX, dY, P: np.column_stack([P[0] + dX, P[6] + dY])
        dFdWdP = lambda dX, dY, dfdx, dfdy: np.column_stack([dfdx, dfdy])
        SFPVec2Mat = lambda P: np.array([[1, 0, 0], [0, 1, 0], [P[0], P[6], 1]])
        Mat2SFPVec = lambda W: np.array([W[2, 0], 0, 0, 0, 0, 0, W[2, 1], 0, 0, 0, 0, 0])
        StopCrit = lambda dP, Zeta: np.sqrt(np.sum((dP * np.array([1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]))**2))

    elif SFOrder == 1:  # First order SF
        W = lambda dX, dY, P: np.column_stack([P[0] + P[2]*dY + dX*(P[1] + 1), P[6] + P[7]*dX + dY*(P[8] + 1)])
        dFdWdP = lambda dX, dY, dfdx, dfdy: np.column_stack([dfdx, dfdx*dX, dfdx*dY, dfdy, dfdy*dX, dfdy*dY])
        SFPVec2Mat = lambda P: np.array([[P[1] + 1, P[7], 0], [P[2], P[8] + 1, 0], [P[0], P[6], 1]])
        Mat2SFPVec = lambda W: np.array([W[2, 0], W[0, 0] - 1, W[1, 0], 0, 0, 0, W[2, 1], W[0, 1], W[1, 1] - 1, 0, 0, 0])
        StopCrit = lambda dP, Zeta: np.sqrt(np.sum((dP * np.array([1, Zeta, Zeta, 0, 0, 0, 1, Zeta, Zeta, 0, 0, 0]))**2))

        #print(f"W = {W}, dfdwdp={dFdWdP}, sfpvec2mat={SFPVec2Mat}, Mat2SFPVec={Mat2SFPVec}, StopCrit = {StopCrit}")

    elif SFOrder == 2:  # Second order SF
        W = lambda dX, dY, P: np.column_stack([
            P[0] + P[2]*dY + P[3]*dX**2*(1/2) + P[5]*dY**2*(1/2) + dX*(P[1] + 1) + P[4]*dX*dY,
            P[6] + P[7]*dX + P[9]*dX**2*(1/2) + P[11]*dY**2*(1/2) + dY*(P[8] + 1) + P[10]*dX*dY
        ])
        dFdWdP = lambda dX, dY, dfdx, dfdy: np.column_stack([
            dfdx, dfdx*dX, dfdx*dY, (dfdx*dX**2)/2, dfdx*dX*dY, (dfdx*dY**2)/2,
            dfdy, dfdy*dX, dfdy*dY, (dfdy*dX**2)/2, dfdy*dX*dY, (dfdy*dY**2)/2
        ])
        SFPVec2Mat = lambda P: np.array([
            [P[1]*2 + P[0]*P[3] + P[1]**2 + 1, P[0]*P[9]*1/2 + P[3]*P[6]*(1/2) + P[7]*(P[1]*2 + 2)*1/2, P[6]*P[9] + P[7]**2],
            [P[3]*1/2, P[9]*1/2, 0],
            [P[0]*P[4]*2 + P[2]*(P[1]*2 + 2), P[1] + P[8] + P[1]*P[8] + P[2]*P[7] + P[0]*P[10] + P[4]*P[6] + 1, P[6]*P[10]*2.0 + P[7]*(P[8] + 1)*2],
            [P[4], P[10], 0],
            [P[0]*P[5] + P[2]**2, P[0]*P[11]*1/2 + P[5]*P[6]*1/2 + P[2]*(P[8] + 1), P[8]*2 + P[6]*P[11] + P[8]**2 + 1],
            [P[5]*1/2, P[11]*1/2, 0]
        ])
        Mat2SFPVec = lambda W: np.array([W[5, 0], W[3, 0] - 1, W[4, 0], W[0, 1]*2, W[1, 1], W[2, 1]*2, W[5, 1], W[3, 1], W[4, 1] - 1, W[0, 2]*2, W[1, 2], W[2, 2]*2])
        StopCrit = lambda dP, Zeta: np.sqrt(np.sum((dP * np.array([1, Zeta, Zeta, 0.5*Zeta**2, Zeta**2, 0.5*Zeta**2, 1, Zeta, Zeta, 0.5*Zeta**2, Zeta**2, 0.5*Zeta**2]))**2))

    else:
        raise ValueError("SFOrder must be 0, 1, or 2")

    return W, dFdWdP, SFPVec2Mat, Mat2SFPVec, StopCrit

In [105]:
import numpy as np

def AddGridFormat(PD):
    for d in range(len(PD)):
        # Determine vectors to relate the subset position to its position within the grid matrices
        XindicesUnique = np.sort(np.ceil(np.unique(PD[d]['Xos'][0, :])).astype(int))
        Xindices = np.full(XindicesUnique[-1], np.nan)
        Xindices[XindicesUnique - 1] = np.arange(1, len(XindicesUnique) + 1)

        YindicesUnique = np.sort(np.ceil(np.unique(PD[d]['Xos'][1, :])).astype(int))
        Yindices = np.full(YindicesUnique[-1], np.nan)
        Yindices[YindicesUnique - 1] = np.arange(1, len(YindicesUnique) + 1)

        # Initialize grid matrices
        grid_shape = (len(YindicesUnique), len(XindicesUnique))
        PD[d]['POSX'] = np.full(grid_shape, np.nan)
        PD[d]['POSY'] = np.full(grid_shape, np.nan)
        PD[d]['UX'] = np.full(grid_shape, np.nan)
        PD[d]['UY'] = np.full(grid_shape, np.nan)
        CheckElements = np.zeros(grid_shape)

        # Convert the data to the format of grid matrices
        for q in range(PD[d]['Xow'].shape[1]):
            y_idx = int(Yindices[int(np.ceil(PD[d]['Xos'][1, q])) - 1] - 1)
            x_idx = int(Xindices[int(np.ceil(PD[d]['Xos'][0, q])) - 1] - 1)

            PD[d]['POSX'][y_idx, x_idx] = PD[d]['Xow'][0, q]
            PD[d]['POSY'][y_idx, x_idx] = PD[d]['Xow'][1, q]
            PD[d]['UX'][y_idx, x_idx] = PD[d]['Uw'][0, q]
            PD[d]['UY'][y_idx, x_idx] = PD[d]['Uw'][1, q]
            CheckElements[y_idx, x_idx] = PD[d]['Xos'][0, q]

        # Set the values of the elements of the grid matrices that do not correspond to an analysed subset to NaN
        mask = CheckElements == 0
        PD[d]['POSX'][mask] = np.nan
        PD[d]['POSY'][mask] = np.nan
        PD[d]['UX'][mask] = np.nan
        PD[d]['UY'][mask] = np.nan

    return PD

In [106]:
import numpy as np
import cv2

def CSTrans(n, PD, WorldCTs, ImgCTs, rho):
    if WorldCTs is not None and ImgCTs is not None:
        # Estimate camera parameters
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(WorldCTs, ImgCTs, PD[0]['ImgSize'], None, None)
        CamParams = {
            'IntrinsicMatrix': mtx,
            'RadialDistortion': dist[:2],
            'TangentialDistortion': dist[2:4]
        }
    else:
        CamParams = {
            'IntrinsicMatrix': np.eye(3),
            'RadialDistortion': np.zeros(2),
            'TangentialDistortion': np.zeros(2)
        }
        WorldCTs = PD[0]['Xos'].T
        ImgCTs = PD[0]['Xos'].T

    # Get extrinsics for the last frame
    _, rvec, tvec = cv2.solvePnP(WorldCTs, ImgCTs[:, :, -1], CamParams['IntrinsicMatrix'],
                                 np.concatenate((CamParams['RadialDistortion'], CamParams['TangentialDistortion'])))
    R, _ = cv2.Rodrigues(rvec)
    T = tvec.flatten()

    # Equation (6)
    Tspec = T - np.dot([0, 0, rho], R)

    for d in range(n):
        # Equation (24)
        Xds = np.vstack((PD[d]['Xos'][0, :] + PD[d]['P'][0, :],
                         PD[d]['Xos'][1, :] + PD[d]['P'][6, :]))

        # Undistort points
        Xos_undistorted = cv2.undistortPoints(PD[d]['Xos'].T.reshape(-1, 1, 2),
                                              CamParams['IntrinsicMatrix'],
                                              np.concatenate((CamParams['RadialDistortion'], CamParams['TangentialDistortion'])))
        Xds_undistorted = cv2.undistortPoints(Xds.T.reshape(-1, 1, 2),
                                              CamParams['IntrinsicMatrix'],
                                              np.concatenate((CamParams['RadialDistortion'], CamParams['TangentialDistortion'])))

        # Transform points to world coordinates
        PD[d]['Xow'] = cv2.projectPoints(Xos_undistorted, rvec, Tspec.reshape(3, 1),
                                         CamParams['IntrinsicMatrix'], None)[0].reshape(-1, 2).T
        Xdw = cv2.projectPoints(Xds_undistorted, rvec, Tspec.reshape(3, 1),
                                CamParams['IntrinsicMatrix'], None)[0].reshape(-1, 2).T

        # Equation (26)
        PD[d]['Uw'] = Xdw - PD[d]['Xow']
        PD[d]['CamParams'] = CamParams

    return PD

In [37]:
ProcData[0]

{'ImgName': '/content/drive/MyDrive/DIC/correctedImage/Sample3/Sample3 Reference.tif',
 'ImgSize': (512, 512),
 'ImgFilt': [0.4, 5],
 'SubSize': array([41., 41., 41., ..., 41., 41., 41.]),
 'SubShape': array([['Circle'],
        ['Circle'],
        ['Circle'],
        ...,
        ['Circle'],
        ['Circle'],
        ['Circle']], dtype='<U6'),
 'SFOrder': array([[1],
        [1],
        [1],
        ...,
        [1],
        [1],
        [1]]),
 'Xos': array([[ 26.,  31.,  36., ..., 476., 481., 486.],
        [ 26.,  26.,  26., ..., 486., 486., 486.]]),
 'P': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'C': array([1., 1., 1., ..., 1., 1., 1.]),
 'StopVal': array([0.0001, 0.0001, 0.0001, ..., 0.0001, 0.0001, 0.0001]),
 'Iter': array([0., 0., 0., ..., 0., 0., 0.])}